# Gerekli Kutuphanelerin Import Edilmesi

In [1]:
import os
import sys
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from sklearn import preprocessing
from sklearn.preprocessing import scale
import tensorflow as tf
from tensorflow import keras

# Data Setinden Cekilecek Veriler Icin Gerekli Variable Tanımlamaları

In [2]:
# Kaç adet train edeceğimiz veri olduğunu len_trainData değişkenine
# Kaç adet test edeceğimiz veri olduğunu len_testData değişkenine atıyoruz
# Ben 194 data'ya sahip olduğum ve test oranının 20% olmasını istediğim için aşağıdaki şekilde atadım
len_trainData = int( 194 * 0.8)
len_testData = int( 194 * 0.2)
# Görsellerin Boyutu
image_sizeX = 128
image_sizeY = 128

# Train edilecek görselleri daha sonra içine atamak için oluşturduğum boş listeler
train_data = []
train_mask1 = []
train_mask2 = []

# Test edilecek görselleri daha sonra içine atamak için oluşturduğum boş listeler
test_data = []
test_mask1 = []
test_mask2 = []

# Dataseti Boş Listelere Yükleme

In [3]:
for i in range(len_trainData - 1):
    # matplotlib.image kütüphanesinin içinde gelen imread ile dosyaları okuyup geçici bir değişkende saklıyoruz
    x = mpimg.imread('./dataset/data/%d.png'%(i+1))
    
    # Bu görsel bize normal bir liste olarak geldiği için reshape ederek 128x128 hale getiriyoruz. Normalde 16384 (128*128) değişken içeren bir listeydi
    x = x.reshape(image_sizeX,image_sizeY,1)
    
    # bu görseli listeye atıyoruz.
    train_data.append(x)
    
    # Aynı işlemi masklar için de yapıyoruz
    
    m1 = mpimg.imread('./dataset/mask1/%d.png'%(i+1))
    m1 = m1.reshape(image_sizeX,image_sizeY,1)
    train_mask1.append(m1)
    m2 = mpimg.imread('./dataset/mask2/%d.png'%(i+1))
    m2 = m2.reshape(image_sizeX,image_sizeY,1)
    train_mask2.append(m2)

# Test verileri için yeni bir foreach loop'u
# Tüm verilerin sayısı len_trainData + len_testData toplamına eşit ve en son (len_trainData-1). değeri zaten train data'da atadık
for i in range(len_trainData, len_trainData + len_testData):
    x = mpimg.imread('./dataset/data/%d.png'%(i+1))
    x = x.reshape(image_sizeX,image_sizeY,1)
    test_data.append(x)
    
    m1 = mpimg.imread('./dataset/mask1/%d.png'%(i+1))
    m1 = m1.reshape(image_sizeX,image_sizeY,1)
    test_mask1.append(m1)
    
    m2 = mpimg.imread('./dataset/mask2/%d.png'%(i+1))
    m2 = m2.reshape(image_sizeX,image_sizeY,1)
    test_mask2.append(m2)

# Verileri Numpy Arraylere Çevirme

In [4]:
# Şimdi ise gelen veriler çok büyük matrislerden oluştuğu için bunlarla daha rahat çalışma amacıyla numpy 
train_data = np.array(train_data)
train_mask1 = np.array(train_mask1)
train_mask2 = np.array(train_mask2)
test_data = np.array(test_data)
test_mask1 = np.array(test_mask1)
test_mask2 = np.array(test_mask2)

# Unet Modeli Katmanlarının Tanımlanması

In [5]:
# multi_scale fonksiyonu, çeşitli filtre boyutlarında konvolüsyon katmanlarını birleştirerek çok ölçekli bir yapı oluşturur.
# Bu yapı, ağın farklı ölçeklerdeki özellikleri öğrenmesine olanak tanır.
def multi_scale(x, filters, padding="same", strides=1):
    
    # Giriş verisi üzerine 1x1 boyutlu konvolüsyon uygulanarak bir konvolüsyon katmanı oluşturulur.
    c1 = keras.layers.Conv2D(filters, (1,1), padding=padding, strides=strides, activation="relu")(x)
    
    # Giriş verisi üzerine 3x3 boyutlu konvolüsyon uygulanarak bir konvolüsyon katmanı oluşturulur.
    c3 = keras.layers.Conv2D(filters, (3,3), padding=padding, strides=strides, activation="relu")(x)
    
    # Giriş verisi üzerine 5x5 boyutlu konvolüsyon uygulanarak bir konvolüsyon katmanı oluşturulur.
    c5 = keras.layers.Conv2D(filters, (5,5), padding=padding, strides=strides, activation="relu")(x)
    
    # Giriş verisi üzerine 7x7 boyutlu konvolüsyon uygulanarak bir konvolüsyon katmanı oluşturulur.
    c7 = keras.layers.Conv2D(filters, (7,7), padding=padding, strides=strides, activation="relu")(x)
    
    # Oluşturulan tüm konvolüsyon katmanları birleştirilerek çok ölçekli bir yapı elde edilir.
    c = keras.layers.Concatenate()([c1, c3, c5, c7])
    return c

# down_block2 fonksiyonu, Unet mimarisindeki indirgeme (downsampling) bloğunu tanımlar.
# Bu blok, konvolüsyon katmanları ve max pooling katmanları içerir.
def down_block2(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    
    # Filtre sayısının dörtte biri kadar bir değer hesaplanır.
    f = int(filters/4)
    
    # multi_scale fonksiyonu kullanılarak çok ölçekli bir yapı elde edilir.
    c = multi_scale(x, f)
    
    # Oluşturulan yapı üzerine konvolüsyon uygulanarak bir konvolüsyon katmanı oluşturulur.
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    
    # Max pooling işlemi uygulanarak indirgeme yapılan bir katman oluşturulur.
    p = keras.layers.MaxPool2D((2, 2), (2, 2))(c)
    return c, p

# up_block2 fonksiyonu, Unet mimarisindeki genişletme (upsampling) bloğunu tanımlar.
# Bu blok, birleştirme (concatenation) ve konvolüsyon katmanları içerir.
def up_block2(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    # Filtre sayısının dörtte biri kadar bir değer hesaplanır.
    f = int(filters/4)
    
    # Upsampling işlemi uygulanarak boyutları genişletilmiş bir veri oluşturulur.
    us = keras.layers.UpSampling2D((2, 2))(x)
    
    # Oluşturulan veri üzerine konvolüsyon uygulanarak bir konvolüsyon katmanı oluşturulur.
    us = keras.layers.Conv2D(filters, (2, 2), padding='same', strides=1, activation="relu")(us)
    
    # Önceki katman ile atlanan bağlantılar birleştirilerek yeni bir yapı oluşturulur.
    c = keras.layers.Concatenate()([us, skip])
    
    # Oluşturulan yapı üzerine multi_scale fonksiyonu kullanılarak çok ölçekli bir yapı elde edilir.
    c = multi_scale(c, f)
    
    # Oluşturulan yapı üzerine konvolüsyon uygulanarak bir konvolüsyon katmanı oluşturulur.
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

# bottleneck2 fonksiyonu, Unet mimarisindeki darboğaz (bottleneck) bloğunu tanımlar.
# Bu blok, çok ölçekli konvolüsyon katmanları ve standart bir konvolüsyon katmanı içerir.
def bottleneck2(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    # Filtre sayısının dörtte biri kadar bir değer hesaplanır.
    f = int(filters/4)
    
    # multi_scale fonksiyonu kullanılarak çok ölçekli bir yapı elde edilir.
    c = multi_scale(x, f)
    
    # Oluşturulan yapı üzerine konvolüsyon uygulanarak bir konvolüsyon katmanı oluşturulur.
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c


# DMRFUnet - 2 Aşamalı Modelin Tanımlanması

In [6]:
# Bu fonksiyon, Unet mimarisine dayalı olarak özel bir iki aşamalı modeli tanımlar.
# İlk aşamada bir maske tahmin edilir, ikinci aşamada ise bu maske giriş görüntüsü ile birleştirilerek ikinci bir maske tahmin edilir.
def DMRF_UNet():

    f0 = 64
    f = [f0, f0*2, f0*4, f0*8, f0*16]
    inputs = keras.layers.Input((image_sizeX, image_sizeY, 1))
    
    #### model_1
    p0 = inputs
    c1, p1 = down_block2(p0, f[0])
    c2, p2 = down_block2(p1, f[1])
    c3, p3 = down_block2(p2, f[2])
    c4, p4 = down_block2(p3, f[3])
    
    bn = bottleneck2(p4, f[4])
    
    u1 = up_block2(bn, c4, f[3])
    u2 = up_block2(u1, c3, f[2])
    u3 = up_block2(u2, c2, f[1])
    u4 = up_block2(u3, c1, f[0])
    
    output_1 = keras.layers.Conv2D(1, (1, 1), padding="same", activation="relu")(u4)
    model_1 = tf.keras.Model(inputs, output_1)

    #### model_2
    pp0 = keras.layers.Concatenate()([output_1, inputs])
    cc1, pp1 = down_block2(pp0, f[0])
    cc2, pp2 = down_block2(pp1, f[1])
    cc3, pp3 = down_block2(pp2, f[2])
    cc4, pp4 = down_block2(pp3, f[3])
    
    bnn = bottleneck2(pp4, f[4])
    
    uu1 = up_block2(bnn, cc4, f[3])
    uu2 = up_block2(uu1, cc3, f[2])
    uu3 = up_block2(uu2, cc2, f[1])
    uu4 = up_block2(uu3, cc1, f[0])

    output_2 = keras.layers.Conv2D(1, (1, 1), padding="same", activation="elu")(uu4)
    model_2 = tf.keras.Model(inputs, [output_1, output_2])

    return model_2

# get_lr_metric fonksiyonu, bir optimizer nesnesi alarak öğrenme oranını döndüren bir fonksiyon oluşturur.
def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr

# DMRF_UNet fonksiyonunu kullanarak bir model oluşturuyoruz
model = DMRF_UNet()

# Bu modelin özetini görüntülüyoruz.
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 128, 128,  │         32 │ input_layer[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 128, 128,  │        160 │ input_layer[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 128, 128,  │        416 │ input_layer[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 128, 128,  │        800 │ input_layer[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 128, 128,  │          0 │ conv2d[0][0],     │
│ (Concatenate)       │ 64)               │            │ conv2d_1[0][0],   │
│                     │                   │            │ conv2d_2[0][0],   │
│                     │                   │            │ conv2d_3[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 128, 128,  │     36,928 │ concatenate[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 64, 64,    │          0 │ conv2d_4[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 64, 64,    │      2,080 │ max_pooling2d[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 64, 64,    │     18,464 │ max_pooling2d[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 64, 64,    │     51,232 │ max_pooling2d[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 64, 64,    │    100,384 │ max_pooling2d[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 64, 64,    │          0 │ conv2d_5[0][0],   │
│ (Concatenate)       │ 128)              │            │ conv2d_6[0][0],   │
│                     │                   │            │ conv2d_7[0][0],   │
│                     │                   │            │ conv2d_8[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 64, 64,    │    147,584 │ concatenate_1[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 32, 32,    │          0 │ conv2d_9[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 95,487,426 (364.26 MB)

 Trainable params: 95,487,426 (364.26 MB)

 Non-trainable params: 0 (0.00 B)

# Modelin Train Edilmesi

In [ ]:
total_epoch = 100
batch_size = 100
alpha = 10
beta = 1
path = './'
model_path = path + 'exp/model.keras'
Adam = keras.optimizers.Adam(learning_rate=1e-4)
lr_metric = get_lr_metric(Adam)
model.compile(optimizer=Adam, loss=['mse', 'mse'], loss_weights=[alpha, beta], metrics=['mae', 'mae'])
model_checkpoint = keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss', verbose=1, save_best_only=True)
lr_checkpoint = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.95, patience=2, min_lr=0)
history = model.fit(x=train_data, y=[train_mask1, train_mask2], batch_size=batch_size, epochs=total_epoch, verbose=2, \
    validation_data=(test_data, [test_mask1, test_mask2]), callbacks=[model_checkpoint])

Epoch 1/100

Epoch 1: val_loss improved from inf to 2.72734, saving model to ./exp/model.keras
2/2 - 212s - 106s/step - conv2d_49_mae: 0.5192 - conv2d_99_mae: 0.0279 - loss: 2.7215 - val_conv2d_49_mae: 0.5192 - val_conv2d_99_mae: 0.0175 - val_loss: 2.7273
Epoch 2/100

Epoch 2: val_loss improved from 2.72734 to 2.13645, saving model to ./exp/model.keras
2/2 - 206s - 103s/step - conv2d_49_mae: 0.5087 - conv2d_99_mae: 0.0141 - loss: 2.5791 - val_conv2d_49_mae: 0.4588 - val_conv2d_99_mae: 0.0093 - val_loss: 2.1365
Epoch 3/100

Epoch 3: val_loss improved from 2.13645 to 1.38688, saving model to ./exp/model.keras
2/2 - 243s - 121s/step - conv2d_49_mae: 0.4465 - conv2d_99_mae: 0.0099 - loss: 1.9851 - val_conv2d_49_mae: 0.3674 - val_conv2d_99_mae: 0.0166 - val_loss: 1.3869
Epoch 4/100

Epoch 4: val_loss did not improve from 1.38688
2/2 - 183s - 91s/step - conv2d_49_mae: 0.3084 - conv2d_99_mae: 0.0139 - loss: 0.9410 - val_conv2d_49_mae: 0.5056 - val_conv2d_99_mae: 0.0107 - val_loss: 3.4377
Epoc

# Modelin Test Verileri Üzerindeki Performansının Değerlendirilmesi

In [ ]:
model.load_weights(model_path)
model.evaluate(x=test_data, y=[test_mask1, test_mask2], batch_size=batch_size)
[test_pred1, test_pred2] = model.predict(test_data)
for i in range(len(test_data)):
    pred1 = test_pred1[i].reshape(image_sizeX, image_sizeY) * 255
    pred1 = Image.fromarray(pred1)
    pred1.convert('L').save(path + 'exp/visual/1pred_%d.png'%(i+1))
    pred2 = test_pred2[i].reshape(image_sizeX, image_sizeY) * 255
    pred2 = Image.fromarray(pred2)
    pred2.convert('L').save(path + 'exp/visual/2pred_%d.png'%(i+1))

# Modelin Verilen Index Ile Bir Örneğe Uygulanması

In [ ]:
index = 0  # Örnek bir indeks seçilebilir
input_image = test_data[index]
predicted_mask1, predicted_mask2 = model.predict(np.expand_dims(input_image, axis=0))

plt.figure(figsize=(10, 5))

# Giriş görüntüsü
plt.subplot(1, 2, 1)
plt.imshow(np.squeeze(input_image), cmap='gray')
plt.title('Input Image')
plt.axis('off')

# Tahmin edilen maskelerden biri
plt.subplot(1, 2, 2)
plt.imshow(np.squeeze(predicted_mask1), cmap='gray')
plt.title('Predicted Mask 1')
plt.axis('off')

plt.show()